In [1]:
from CSVModifier import aggregate_high_freq_to_low, add_factors, clean_outliers
import numpy as np
import pandas as pd


from Util import fetch_stock_data_finnhub_paginated

In [2]:
SYMBOL = 'SPY'
START_DATE = '2020-01-01'
END_DATE = '2025-05-07'
FINHUB_TOKEN = 'cvop3lhr01qihjtq3uvgcvop3lhr01qihjtq3v00'
HIFRREQ = '1'
LOFREQ = '5'

In [ ]:
# 请求数据
df_hiFreq = fetch_stock_data_finnhub_paginated(
    symbol= SYMBOL,
    start_date=START_DATE,
    end_date=END_DATE,
    interval=HIFRREQ, # 单位为分钟
    token=FINHUB_TOKEN,
    chunk_days=25,
    verbose=True
)

df_loFreq = fetch_stock_data_finnhub_paginated(
    symbol= SYMBOL,
    start_date=START_DATE,
    end_date=END_DATE,
    interval=LOFREQ, # 单位为分钟
    token=FINHUB_TOKEN,
    chunk_days=25,
    verbose=True
)


In [4]:
path_lo = f"{SYMBOL}_{START_DATE}_{END_DATE}_{LOFREQ}m_raw.csv"
path_hi = f"{SYMBOL}_{START_DATE}_{END_DATE}_{HIFRREQ}m_raw.csv"
df_loFreq.to_csv(path_lo, index=False)
df_hiFreq.to_csv(path_hi, index=False)

In [11]:
# 检查数据
df_loFreq.shape
# df_hiFreq.tail(2)

(242741, 45)

In [18]:
# 把高频转换成低频集合
df_hiFreq_agg = aggregate_high_freq_to_low (df_hiFreq, freq='5min')

# 计算因子
df_loFreq_factored = add_factors (df_loFreq)

# # 拼接
df_merged = pd.merge (df_loFreq_factored, df_hiFreq_agg, how='left', on='timestamp')
df_merged = clean_outliers(df_merged, z_thresh=10, show_msg=True)

检测到 0 个异常值在 open 列
检测到 0 个异常值在 high 列
检测到 0 个异常值在 low 列
检测到 0 个异常值在 close 列
检测到 290 个异常值在 volume 列
检测到 0 个异常值在 vwap 列
检测到 0 个异常值在 EMA5_open 列
检测到 0 个异常值在 EMA10_open 列
检测到 0 个异常值在 EMA20_open 列
检测到 0 个异常值在 EMA5_high 列
检测到 0 个异常值在 EMA10_high 列
检测到 0 个异常值在 EMA20_high 列
检测到 0 个异常值在 EMA5_low 列
检测到 0 个异常值在 EMA10_low 列
检测到 0 个异常值在 EMA20_low 列
检测到 0 个异常值在 EMA5_close 列
检测到 0 个异常值在 EMA10_close 列
检测到 0 个异常值在 EMA20_close 列
检测到 0 个异常值在 RSI_open 列
检测到 81 个异常值在 MACD_value_open 列
检测到 55 个异常值在 MACD_signal_open 列
检测到 103 个异常值在 MACD_histogram_open 列
检测到 154 个异常值在 ROC_open 列
检测到 0 个异常值在 RSI_high 列
检测到 86 个异常值在 MACD_value_high 列
检测到 59 个异常值在 MACD_signal_high 列
检测到 113 个异常值在 MACD_histogram_high 列
检测到 165 个异常值在 ROC_high 列
检测到 0 个异常值在 RSI_low 列
检测到 81 个异常值在 MACD_value_low 列
检测到 58 个异常值在 MACD_signal_low 列
检测到 111 个异常值在 MACD_histogram_low 列
检测到 149 个异常值在 ROC_low 列
检测到 0 个异常值在 RSI_close 列
检测到 80 个异常值在 MACD_value_close 列
检测到 52 个异常值在 MACD_signal_close 列
检测到 103 个异常值在 MACD_histogram_close 列
检测到 159 个异常值在 ROC_close 

/Users/ziwenchen/TockFormer/CSVModifier.py:206: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1549641.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  series[i] = (left + right) / 2


In [20]:
# 检查数值
# df_merged[df_merged['close_volatility'] == 0]

df_merged.shape

(240462, 50)

In [21]:
import re
path = f"{SYMBOL}_{START_DATE}_{END_DATE}_{LOFREQ}m.csv"
# path = re.sub(r'[^\w\-]', '', path)
df_merged.to_csv(path, index=False)